In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import pandas as pd
import numpy as np
import gzip
import json
import math
from datetime import datetime
from IPython.display import display
%matplotlib inline

In [2]:
from helper.manage_orderbooks_v2 import extract_orderbooks_for_one_currencypair, load_orderbook_snapshot
from helper.orderbook_container import OrderbookContainer
from helper.orderbook_trader import OrderbookTradingSimulator

In [3]:
filename = '3000orderbooks'
data = load_orderbook_snapshot(filename, last_line=100)

100%|██████████| 100/100 [00:06<00:00, 15.01it/s]

Loaded 100 orderbooks from file '3000orderbooks'.


In [4]:
orderbook = data[0]
print(orderbook)
orderbook2 = data[1]
print(orderbook2)

print("\nCompare two orderbooks_")
diff = orderbook.compare_with(other=orderbook2)

display(diff.head())
print(diff)

OrderbookContainer from 2016-11-08T10:00
  499 bids (best: 705.0)
  407 asks (best: 705.45)
  kind: 'orderbook'
OrderbookContainer from 2016-11-08T10:01
  499 bids (best: 703.8)
  406 asks (best: 705.45)
  kind: 'orderbook'

Compare two orderbooks_


,Amount,Type
678.31,-0.510201,bid
674.59,0.546871,bid
673.62,-0.545428,bid
705.45,-1.876972,ask
706.41,-0.045817,ask
706.81,-7.529719,ask


OrderbookContainer from 2016-11-08T10:01
  23 bids (best: 673.62)
  27 asks (best: 705.45)
  kind: 'diff'


In [5]:
vol = 40
print("Current MarketOrderPrice for {} shares: {}".format(vol, orderbook.get_current_price(vol)))

Current MarketOrderPrice for 40 shares: 28358.1256107


In [6]:
display(orderbook.head(2))
display(orderbook.tail(2))

,Amount,Type
703.700000,0.001250,bid
705.000000,0.158232,bid
705.224976,NaN,center
705.450000,3.172181,ask
706.170000,0.052838,ask


,Amount,Type
614.000000,1.838163,bid
614.320000,0.034763,bid
705.224976,NaN,center
810.000000,0.286798,ask
811.000000,0.000100,ask


In [10]:
ots = OrderbookTradingSimulator()
vol = 40
T=9
P = 2
for e in range(T):
    print("epoch {}".format(e))
    res = ots.trade_timespan(orderbooks=data[P*e:P*(e+1)], volume=vol, limit=707.5, timespan=P, verbose=True)
    vol = ots.history.volume_left.values[-1]
# res = ots.trade_timespan(orderbooks=data[2:4], volume=40, limit=705.5, timespan=2)

display(ots.history)
display(ots.buy_history)
display(res.head(5))

epoch 0
Traded 21.6770/40.0000 shares for -15317.2981269, 18.3230 shares left
epoch 1
Traded 1.8090/18.3230 shares for -1276.12574571, 16.5141 shares left
epoch 2
Traded 0.0000/16.5141 shares for 0, 16.5141 shares left
epoch 3
Traded 0.0000/16.5141 shares for -0.00846, 16.5140 shares left
epoch 4
Traded 8.0784/16.5140 shares for -5712.63775969, 8.4356 shares left
epoch 5
Traded 0.0000/8.4356 shares for 0, 8.4356 shares left
epoch 6
Traded 1.5367/8.4356 shares for -1086.67292909, 6.8990 shares left
epoch 7
Traded 6.2908/6.8990 shares for -4448.57031287, 0.6082 shares left
epoch 8
Traded 0.0022/0.6082 shares for -1.527578, 0.6061 shares left


,ASK,BID,CENTER,LIMIT,SPREAD,TIMESPAN,VOLUME,avg,cashflow,cost,cost_avg,high,low,volume_left,volume_traded
2016-11-08T10:00,705.45,705.00,705.224976,707.5,0.45,2,40.000000,706.61535,-15317.298127,30.139141,1.390374,707.50,705.450000,18.323004,21.676996
2016-11-08T10:02,705.45,703.80,704.624678,707.5,1.65,2,18.323004,705.45000,-1276.125746,0.407058,0.225024,705.45,705.450000,16.514051,1.808953
2016-11-08T10:04,707.52,703.80,705.658366,707.5,3.72,2,16.514051,0.00000,0.000000,0.000000,0.000000,0.00,inf,16.514051,0.000000
2016-11-08T10:06,707.52,703.80,705.658366,707.5,3.72,2,16.514051,705.00000,-0.008460,-0.000003,-0.224976,705.00,705.000000,16.514039,0.000012
2016-11-08T10:08,707.52,704.11,705.813627,707.5,3.41,2,16.514039,707.15000,-5712.637760,15.551106,1.925024,707.15,707.150000,8.435643,8.078396
2016-11-08T10:10,707.52,704.11,705.813627,707.5,3.41,2,8.435643,0.00000,0.000000,0.000000,0.000000,0.00,inf,8.435643,0.000000
2016-11-08T10:12,707.52,704.00,705.758537,707.5,3.52,2,8.435643,707.16000,-1086.672929,2.973497,1.935024,707.16,707.160000,6.898971,1.536672
2016-11-08T10:14,707.52,704.00,705.758537,707.5,3.52,2,6.898971,707.16000,-4448.570313,12.172761,1.935024,707.16,707.160000,0.608216,6.290755
2016-11-08T10:16,707.52,704.00,705.758537,707.5,3.52,2,0.608216,707.21204,-1.527578,0.004292,1.987064,707.30,707.100000,0.606056,0.002160


,Amount
705.00,0.000012
705.45,6.858106
706.17,0.052838
706.19,0.645323
706.41,0.045817
706.81,7.529719
706.89,0.105160
707.10,0.000950
707.15,8.078396
707.16,7.827427


,Amount,Type
703.690000,1.700000,bid
703.700000,0.001250,bid
704.090000,1.662600,bid
704.110000,1.752866,bid
704.600000,6.132076,bid
706.058994,NaN,center
707.520000,1.578535,ask
707.600000,0.000110,ask
707.610000,0.002725,ask
707.700000,0.000690,ask
